### Agent Framework - RAG 

In [1]:
! pip install -U agent-framework-azure-ai python-dotenv azure-identity


  Attempting uninstall: azure-ai-projects

    Found existing installation: azure-ai-projects 1.1.0b4

    Uninstalling azure-ai-projects-1.1.0b4:

      Successfully uninstalled azure-ai-projects-1.1.0b4

   ---------------------------------------- 0/2 [azure-ai-projects]
   ---------------------------------------- 0/2 [azure-ai-projects]
   ---------------------------------------- 0/2 [azure-ai-projects]
   ---------------------------------------- 0/2 [azure-ai-projects]
  Attempting uninstall: agent-framework-azure-ai
   ---------------------------------------- 0/2 [azure-ai-projects]
    Found existing installation: agent-framework-azure-ai 1.0.0b251016
   ---------------------------------------- 0/2 [azure-ai-projects]
    Uninstalling agent-framework-azure-ai-1.0.0b251016:
   ---------------------------------------- 0/2 [azure-ai-projects]
      Successfully uninstalled agent-framework-azure-ai-1.0.0b251016
   ---------------------------------------- 0/2 [azure-ai-projects]
   -

In [10]:
import os
import glob
from typing import List

from dotenv import load_dotenv
from azure.identity.aio import AzureCliCredential

from agent_framework import ChatAgent, HostedFileSearchTool, HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient

In [11]:
load_dotenv() 

True

In [ ]:
import os

# 🔁 FILL THESE WITH YOUR REAL VALUES
# From Azure AI Studio → Project → Project details / Project keys
os.environ["AZURE_AI_PROJECT_ENDPOINT"] = "https://smartaiworld751.openai.azure.com/"

# From Azure AI Studio → Deployments → your GPT model deployment name
os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"] = "gpt-5.1"

print("Project endpoint:", os.getenv("AZURE_AI_PROJECT_ENDPOINT"))
print("Model deployment:", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"))


Project endpoint: https://<your-project-name>.projects.azureai.azure.com
Model deployment: <your-model-deployment-name>


In [12]:
# Folder that contains your documents (PDF/MD/TXT/etc.)
DOCS_FOLDER = "./docs"

In [13]:
async def create_vector_store_from_folder(
    client: AzureAIAgentClient,
    folder_path: str,
    vector_store_name: str = "rag_file_search_store",
) -> HostedVectorStoreContent:
    """
    Upload all files in `folder_path` and create a vector store over them.
    Returns a HostedVectorStoreContent you can pass into HostedFileSearchTool.
    """
    pattern = os.path.join(folder_path, "*")
    file_paths: List[str] = [p for p in glob.glob(pattern) if os.path.isfile(p)]

    if not file_paths:
        raise ValueError(f"No files found in folder: {folder_path}")

    print(f"📄 Found {len(file_paths)} files. Uploading to Azure AI project...")

    file_ids: List[str] = []
    for path in file_paths:
        print(f"- Uploading {os.path.basename(path)}")
        uploaded = await client.project_client.agents.files.upload_and_poll(
            file_path=path,
            purpose="assistants",  # required by the API
        )
        file_ids.append(uploaded.id)

    print("🧠 Creating vector store...")
    vector_store = await client.project_client.agents.vector_stores.create_and_poll(
        file_ids=file_ids,
        name=vector_store_name,
    )

    print(f"✅ Vector store created: {vector_store.id}")
    return HostedVectorStoreContent(vector_store_id=vector_store.id)


In [14]:
async def run_rag_query(question: str, folder_path: str = DOCS_FOLDER):
    """
    1. Creates Azure credential + client
    2. Builds a vector store from all docs in folder
    3. Creates a ChatAgent with HostedFileSearchTool (RAG)
    4. Asks the question and streams the answer
    """

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # 1. Create vector store from your folder
        vector_store = await create_vector_store_from_folder(
            chat_client,
            folder_path=folder_path,
            vector_store_name="rag_file_search_store",
        )

        # 2. File search tool bound to this vector store
        file_search_tool = HostedFileSearchTool(
            inputs=[vector_store],   # Important: pass as list
            max_results=10,
            description="Search across uploaded project documents",
        )

        # 3. Create the agent (like your PythonRAGDemo)
        agent: ChatAgent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions=(
                "You are an AI assistant that helps people find information in a set of documents.\n"
                "Use the File Search tool to look up relevant information from the files when needed.\n"
                "If you don't know the answer, say you don't know. Do NOT make up answers."
            ),
            tools=[file_search_tool],
            tool_choice="auto",  # Let the model decide when to call file_search
        )

        print("🤖 Agent created. Asking your question...\n")
        print(f"Q: {question}\n")
        print("A: ", end="", flush=True)

        # 4. Ask the question with streaming response
        async for chunk in agent.run_stream(question):
            if chunk.text:
                print(chunk.text, end="", flush=True)

        print()  # final newline


In [15]:
# Example questions:
# "What is GraphRAG?"
# "Summarize the main ideas from the documents."
# "What are the key risks mentioned in these files?"

question = "What is GraphRAG and how is it used in these documents?"
await run_rag_query(question, folder_path=DOCS_FOLDER)


ServiceInitializationError: Azure AI model deployment name is required. Set via 'model_deployment_name' parameter or 'AZURE_AI_MODEL_DEPLOYMENT_NAME' environment variable.